In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statistics 
import os
import warnings
import nltk
import string
import collections
import matplotlib.cm as cm
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
from collections import Counter
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
from textblob import TextBlob,Word,Blobber
from sklearn.metrics import f1_score, accuracy_score

nltk.download('stopwords')
warnings.filterwarnings('ignore')
filepath = os.path.join(os.getcwd(),'CSV_Files','ninja_reports.xls')
df = pd.read_excel('ninja_reports2.xls',sheet_name='ninja_reports')

df_w = pd.read_csv("ward_details.csv")

[nltk_data] Downloading package stopwords to /home/shriya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
#files_pos = os.listdir('pos.txt')
#files_pos = open("pos.txt", "r")
#files_neg = open("neg.txt", "r")
training_set = open('training_set.txt', 'r')

all_words = []
documents = []

from nltk.corpus import stopwords
import re

stop_words = list(set(stopwords.words('english')))

#  j is adject, r is adverb, and v is verb
allowed_word_types = ["J","R","V"]
#allowed_word_types = ["J"]
count_p = count_n = 0
for p in  training_set:
    
    # create a list of tuples where the first element of each tuple is a review
    # the second element is the label
    positive = float(p.split(',')[1])
    if positive > 0.4:
        documents.append( (p, "pos") )
        count_p += 1
    else:
        documents.append( (p, "neg") )
        count_n += 1
    
    # remove punctuations
    cleaned = re.sub(r'[^(a-zA-Z)\s]','', p)
    
    # tokenize 
    tokenized = word_tokenize(cleaned)
    
    # remove stopwords 
    stopped = [w for w in tokenized if not w in stop_words]
    
    # parts of speech tagging for each word 
    pos = nltk.pos_tag(stopped)
    
    # make a list of  all adjectives identified by the allowed word types list above
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

    
#print(all_words)
#print(documents)
#print(count_p, count_n)

In [24]:
len(all_words)

108

In [39]:

# pickling the list documents to save future recalculations 
filename = "pickled_algos/documents.pickle"
os.makedirs(os.path.dirname(filename), exist_ok=True)
save_documents = 'documents'
with open("pickled_algos/documents.pickle","wb") as f:
    pickle.dump(save_documents, f)

# creating a frequency distribution of each adjectives. 
BOW = nltk.FreqDist(all_words)
BOW

# listing the 5000 most frequent words
word_features = list(BOW.keys())[:500]
word_features[0], word_features[-1]

save_word_features = open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [43]:
# function to create a dictionary of features for each review in the list document.
# The keys are the words in word_features 
# The values of each key are either true or false for wether that feature appears in the review or not
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Creating features for each review
featuresets = [(find_features(rev), category) for (rev, category) in documents]

# Shuffling the documents 
random.shuffle(featuresets)
#print(len(featuresets))

training_set = featuresets
#print(len(training_set))
#testing_set = featuresets[40:]
#print( 'training_set :', len(training_set), '\ntesting_set :', len(testing_set))

training_set : 40 
testing_set : 12


In [44]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

classifier.show_most_informative_features(15)

# Printing the most important features 

mif = classifier.most_informative_features()

mif = [a for a,b in mif]
print(mif)

Classifier accuracy percent: 90.47619047619048
Most Informative Features
                dustbins = True              pos : neg    =      6.0 : 1.0
                    next = True              pos : neg    =      3.6 : 1.0
                   right = True              pos : neg    =      2.6 : 1.0
                  thrown = True              pos : neg    =      2.0 : 1.0
                  filled = False             neg : pos    =      2.0 : 1.0
                 plastic = False             neg : pos    =      1.4 : 1.0
                  dumped = False             pos : neg    =      1.3 : 1.0
                  coming = False             neg : pos    =      1.2 : 1.0
                    main = False             neg : pos    =      1.2 : 1.0
                    cows = False             neg : pos    =      1.2 : 1.0
                  eating = False             neg : pos    =      1.2 : 1.0
                  parked = False             neg : pos    =      1.2 : 1.0
                dustbins = 

In [45]:
# getting predictions for the testing set by looping over each reviews featureset tuple
# The first elemnt of the tuple is the feature set and the second element is the label 
ground_truth = [r[1] for r in testing_set]

preds = [classifier.classify(r[0]) for r in testing_set]

from sklearn.metrics import f1_score
f1_score(ground_truth, preds, labels = ['neg', 'pos'], average = 'micro')

0.9047619047619048

In [46]:


from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC



In [47]:
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_clf = SklearnClassifier(MultinomialNB())
MNB_clf.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_clf, testing_set))*100)

BNB_clf = SklearnClassifier(BernoulliNB())
BNB_clf.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BNB_clf, testing_set))*100)

LogReg_clf = SklearnClassifier(LogisticRegression())
LogReg_clf.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogReg_clf, testing_set))*100)

SGD_clf = SklearnClassifier(SGDClassifier())
SGD_clf.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGD_clf, testing_set))*100)

SVC_clf = SklearnClassifier(SVC())
SVC_clf.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_clf, testing_set))*100)

Original Naive Bayes Algo accuracy percent: 90.47619047619048
Most Informative Features
                dustbins = True              pos : neg    =      6.0 : 1.0
                    next = True              pos : neg    =      3.6 : 1.0
                   right = True              pos : neg    =      2.6 : 1.0
                  thrown = True              pos : neg    =      2.0 : 1.0
                  filled = False             neg : pos    =      2.0 : 1.0
                 plastic = False             neg : pos    =      1.4 : 1.0
                  dumped = False             pos : neg    =      1.3 : 1.0
                  coming = False             neg : pos    =      1.2 : 1.0
                    main = False             neg : pos    =      1.2 : 1.0
                    cows = False             neg : pos    =      1.2 : 1.0
                  eating = False             neg : pos    =      1.2 : 1.0
                  parked = False             neg : pos    =      1.2 : 1.0
            

In [48]:


def create_pickle(c, file_name): 
    save_classifier = open(file_name, 'wb')
    pickle.dump(c, save_classifier)
    save_classifier.close()

classifiers_dict = {'ONB': [classifier, 'pickled_algos/ONB_clf.pickle'],
                    'MNB': [MNB_clf, 'pickled_algos/MNB_clf.pickle'],
                    'BNB': [BNB_clf, 'pickled_algos/BNB_clf.pickle'],
                    'LogReg': [LogReg_clf, 'pickled_algos/LogReg_clf.pickle'],
                    'SGD': [SGD_clf, 'pickled_algos/SGD_clf.pickle'], 
                    'SVC': [SVC_clf, 'pickled_algos/SVC_clf.pickle']}




for clf, listy in classifiers_dict.items(): 
    create_pickle(listy[0], listy[1])



In [49]:
from sklearn.metrics import f1_score, accuracy_score
'''acc_scores = {}
for clf, listy in classifiers_dict.items(): 
    # getting predictions for the testing set by looping over each reviews featureset tuple
    # The first elemnt of the tuple is the feature set and the second element is the label 
    acc_scores[clf] = accuracy_score(ground_truth, predictions[clf])
    print(f'Accuracy_score {clf}: {acc_scores[clf]}') '''
ground_truth = [r[1] for r in testing_set]
predictions = {}
f1_scores = {}
for clf, listy in classifiers_dict.items(): 
    # getting predictions for the testing set by looping over each reviews featureset tuple
    # The first elemnt of the tuple is the feature set and the second element is the label 
    predictions[clf] = [listy[0].classify(r[0]) for r in testing_set]
    f1_scores[clf] = f1_score(ground_truth, predictions[clf], labels = ['neg', 'pos'], average = 'micro')
    print(f'f1_score {clf}: {f1_scores[clf]}')

acc_scores = {}
for clf, listy in classifiers_dict.items(): 
    # getting predictions for the testing set by looping over each reviews featureset tuple
    # The first elemnt of the tuple is the feature set and the second element is the label 
    acc_scores[clf] = accuracy_score(ground_truth, predictions[clf])
    print(f'Accuracy_score {clf}: {acc_scores[clf]}')


from nltk.classify import ClassifierI

# Defininig the ensemble model class 

class EnsembleClassifier(ClassifierI):
    
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    # returns the classification based on majority of votes
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    # a simple measurement the degree of confidence in the classification 
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

# Load all classifiers from the pickled files 

# function to load models given filepath
def load_model(file_path): 
    classifier_f = open(file_path, "rb")
    classifier = pickle.load(classifier_f)
    classifier_f.close()
    return classifier


# Original Naive Bayes Classifier
ONB_Clf = load_model('pickled_algos/ONB_clf.pickle')

# Multinomial Naive Bayes Classifier 
MNB_Clf = load_model('pickled_algos/MNB_clf.pickle')


# Bernoulli  Naive Bayes Classifier 
BNB_Clf = load_model('pickled_algos/BNB_clf.pickle')

# Logistic Regression Classifier 
LogReg_Clf = load_model('pickled_algos/LogReg_clf.pickle')

# Stochastic Gradient Descent Classifier
SGD_Clf = load_model('pickled_algos/SGD_clf.pickle')



# Initializing the ensemble classifier 
ensemble_clf = EnsembleClassifier(ONB_Clf, MNB_Clf, BNB_Clf, LogReg_Clf, SGD_Clf)

# List of only feature dictionary from the featureset list of tuples 
feature_list = [f[0] for f in testing_set]

# Looping over each to classify each review
ensemble_preds = [ensemble_clf.classify(features) for features in feature_list]

f1_score(ground_truth, ensemble_preds, average = 'micro')



f1_score ONB: 0.9047619047619048
f1_score MNB: 0.9523809523809523
f1_score BNB: 0.8095238095238095
f1_score LogReg: 0.9047619047619048
f1_score SGD: 0.9523809523809523
f1_score SVC: 0.9523809523809523
Accuracy_score ONB: 0.9047619047619048
Accuracy_score MNB: 0.9523809523809523
Accuracy_score BNB: 0.8095238095238095
Accuracy_score LogReg: 0.9047619047619048
Accuracy_score SGD: 0.9523809523809523
Accuracy_score SVC: 0.9523809523809523


0.9047619047619048

In [50]:
# Function to do classification a given review and return the label a
# and the amount of confidence in the classifications
def sentiment(text):
    feats = find_features(text)
    return ensemble_clf.classify(feats), ensemble_clf.confidence(feats)


In [51]:
# sentiment analysis of reviews of captain marvel found on rotten tomatoes
text_a = '''It's in Besamt Nagar beach. All dustbins are broken and waste is thrown all around.'''
text_b = '''Close to my home, there is a big bin for dumping garbage. But, people are throwing away garbage all around the bi'''
text_c = '''It has a heap of garbage all wet and dry mixed together and there are cows digging through the this heap. Not only this but there is also so much of smell which causes lots of inconvenience for people'''
text_d = '''Lot of garbage right outside the general hospital jayanagar. The garbage has been lying here for days and it stinks too. How can we care for the patients in such an environment?'''
text_e = '''"flowers and children are happy"'''

sentiment(text_a), sentiment(text_b), sentiment(text_c), sentiment(text_d), sentiment(text_e)

(('neg', 1.0), ('neg', 1.0), ('neg', 1.0), ('neg', 1.0), ('neg', 1.0))